In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Character level language model

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size) 


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# Since our language model is character level, our tokens will be individual characters

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # takes a string and outputs a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # takes a list of integers and outputs a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
# Google - SentencePiece
# OpenAI - tiktoken (BPE - bi pair encoding tokeniser)

In [8]:
!pip install git+https://github.com/openai/whisper.git

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\chinn\AppData\Local\Temp\pip-req-build-qwzidkqe'

[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Cloning https://github.com/openai/whisper.git to c:\users\chinn\appdata\local\temp\pip-req-build-qwzidkqe
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
                                              0.0/2.7 MB ? eta -:--:--
     ---                                      0.2/2.7 MB 4.6 MB/s eta 0:00:01
     ----                                     0.3/2.7 MB 4.1 MB/s eta 0:00:01
     -----------                              0.8/2.7 MB 6.1 MB/s eta 0:00:01
     --------------                           1.0/2.7 MB 6.1 MB/s eta 0:00:01
     ------------------                       1.3/2.7 MB 5.3 MB/s eta 0:00:01


In [ ]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
enc.n_vocab

50257

In [10]:
enc.encode("hii there")

[71, 4178, 612]

In [12]:
enc.decode([71, 4178, 612])

'hii there'

In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# For not to overfit i.e., exact memorization
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
# We can't train the entire dataset at a time because of computational reasons
# So we train in chunks of this particular size

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")
# we are taken context_size from 1 to block_size for computational reasons and
# for the transformer network to see the context throughout this range

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
# Now we introduce batches for parallel computation keeping the GPU busy
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    indx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in indx])
    y = torch.stack([data[i+1:i+block_size+1] for i in indx])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 

In [13]:
print(xb) #our input to the model

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, indx, targets=None):
        
        # indx and target are of shape (B, T)
        logits = self.token_embedding_table(indx) # plucks out a row from the embedding table corresponding to the indx, it will be a vocab_size tensor
        # (B, T, C) logits - scores for the next character in the sequence - this is just by seeing itself
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # targets.view(-1)
            loss = F.cross_entropy(logits, targets) # cross entropy expects the second argument to be the number of classes
        # functional.cross_entropy means we don't have to create a module for it
        return logits, loss
     
    def generate(self, indx, max_new_tokens):
        
        # indx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(indx)
            # take the last one because it is the actual next character
            logits = logits[:, -1, :]
            # softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            indx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to get the running sequence
            indx = torch.cat((indx, indx_next), dim=1)
        return indx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(indx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
# Let's train it now
# PyTorch optimizer (instead of Stochastic gradient descent which we used before)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [16]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if steps%1000==0:
       print(loss.item())

4.704006195068359
3.7031264305114746
3.1371781826019287
2.776794672012329
2.5844571590423584
2.5105180740356445
2.531585931777954
2.504757881164551
2.4696717262268066
2.4838879108428955


In [17]:
print(decode(m.generate(indx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y helti


Mathematical tricks for calculation self-attention

In [58]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

# In self-attention the T tokens start talking to each other so that they can figure out what is in the context and make better predictions

torch.Size([4, 8, 2])

In [59]:
# Naive - For now for self-attention we have to calculate the dependency of the current 
# character on all the previous characters in the context 
# We take average as the metric for now - it is extremely lossy - we have lost the spatial arrangement of the tokens

xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [11]:
x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [12]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [13]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a/torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [47]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) by (B, T, C) => (B, T, T) by (B, T, C) => (T, T) by (T, C) for each batch
# torch.allclose(xbow, xbow2, rtol=1e-04, atol=1e-08)

In [62]:
# using Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) 
# better method because it actually represents the contribution of a character towards the average
wei = F.softmax(wei, dim=1)
# print(wei)
xbow3 = wei @ x
torch.allclose(xbow, xbow2, rtol=1e-04, atol=1e-08)

True

In [65]:
# Self-attention - a bit more general

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False) # how much it contributes to the other characters
query = nn.Linear(C, head_size, bias=False) # how much it takes contribution from the other characters
value = nn.Linear(C, head_size, bias=False) # value of the node actually taken for the characters
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float('-inf')) # removing this line makes this an encoder block
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [18]:
# Attention is communication - communicating accross context in a batch but no communication between batches
# this does not have any spatial significance for which we should add positional encodings, this is unlike convolutional where there is significance of positions
# In this case we are not taking the contributions of future characters, but we may need it in some cases like sentimental analysis in which case we do not -inf the later ones - this will be an encoder unlike the present case which is a decoder
# Cross-attention uses keys, queries and values which come from whole separate blocks of x
# scaling attention to unit varaince helps not to sharpen the graph of wei

In [71]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) # *head_size**-0.5

In [68]:
k.var()

tensor(1.0449)

In [69]:
q.var()

tensor(1.0700)

In [70]:
wei.var()

tensor(17.4690)

In [72]:
wei.var()

tensor(0.9957)

In [73]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [74]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [76]:
# all this is attention by dot product
# another one is multi-head attention which is multiple heads of self-attention in parallel

In [2]:
class LayerNorm: # mean and variance along row instead of along column as in batchnorm
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x-xmean)/torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

In [77]:
# We do pre-norm formulation with the Layer norm

In [78]:
# Decoder only transformer

In [80]:
# Drop out layer which we can add right before the residual i.e., before the going back to the original network
# It is a regularization technique

# randomly drops out some neurons during training and because it drop outs randomly in each training it works like a ensemble of a set of subnetworks

In [81]:
# Used when we will scale up the model so that it does not overfit

In [82]:
# We did not implement the cross-attention in the network
# The fact that it uses the triangular mask to mask the attention makes our network a decoder
# We did not implement the encoder because our generation is unconditioned and we just generating based on the dataset we have

In [83]:
# Encoder is needed in cases like translation 
# encodes one language and decodes into another language